# AI Config SDK - Cookbook

This cookbook tests all code from `aiconfig-sdk/SKILL.md` for consuming AI Configs in your Python application.

## Prerequisites
- `LAUNCHDARKLY_SDK_KEY`: SDK key from your project (in `.env` at repo root)
- AI Config created (run `cookbook_aiconfig_create.ipynb` first)

In [31]:
# Install and load environment
%pip install launchdarkly-server-sdk launchdarkly-server-sdk-ai python-dotenv -q

import os
from pathlib import Path
from dotenv import load_dotenv

def find_repo_root(start_path: Path = None) -> Path:
    """Find the repository root by looking for .git directory."""
    current = start_path or Path.cwd()
    for parent in [current] + list(current.parents):
        if (parent / '.git').exists():
            return parent
    return current

# Load .env from repository root
repo_root = find_repo_root()
env_path = repo_root / '.env'
load_dotenv(env_path)
print(f"[OK] Loaded environment from: {env_path}")


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
[OK] Loaded environment from: /Users/ld_scarlett/Documents/Github/agent-skills/.env


---
## Building User Context
From: `SKILL.md` lines 79-96

Note: This is a pure utility function with no SDK dependency, so we define it first.

In [32]:
from ldclient import Context

def build_context(user_id: str, **attributes):
    """Build LaunchDarkly context for targeting."""
    builder = Context.builder(user_id)
    for key, value in attributes.items():
        builder.set(key, value)
    return builder.build()

# Basic context
context = build_context("user-123")

# Context with attributes for targeting
context = build_context(
    "user-123",
    subscription_tier="premium",
    region="us-west"
)
print(f"[OK] build_context defined")

[OK] build_context defined


---
## Basic SDK Setup
From: `SKILL.md` lines 52-75

In [33]:
import os
import ldclient
from ldclient import Context
from ldclient.config import Config
from ldai.client import LDAIClient

def initialize_launchdarkly(sdk_key: str):
    """Initialize LaunchDarkly SDK."""
    config = Config(sdk_key)
    ldclient.set_config(config)

    ld_client = ldclient.get()
    ai_client = LDAIClient(ld_client)

    if not ld_client.is_initialized():
        raise Exception("LaunchDarkly client failed to initialize")

    print(f"[OK] SDK initialized")
    return ld_client, ai_client

SDK_KEY = os.environ.get("LAUNCHDARKLY_SDK_KEY")
ld_client, ai_client = initialize_launchdarkly(SDK_KEY)

[OK] SDK initialized


---
## Verify: initialize_launchdarkly()

In [34]:
# Verify SDK initialized properly
print(f"SDK Initialized: {ld_client.is_initialized()}")
print(f"LD Client type: {type(ld_client).__name__}")
print(f"AI Client type: {type(ai_client).__name__}")

SDK Initialized: True
LD Client type: LDClient
AI Client type: LDAIClient


---
## Consuming Completion Configs
From: `SKILL.md` lines 100-124

In [35]:
from typing import Dict
from ldai.client import AICompletionConfigDefault, ModelConfig, LDMessage, ProviderConfig

# Register fallback configs for specific config keys that need fallbacks
fallback_configs: Dict[str, AICompletionConfigDefault] = {}

def get_completion_config(ai_client, config_key: str, context, variables: dict = None):
    """Get completion-mode AI Config with optional fallback."""
    fallback = fallback_configs.get(config_key, AICompletionConfigDefault(enabled=False))
    config = ai_client.completion_config(config_key, context, fallback, variables or {})
    return config

# Test with config created by aiconfig-create cookbook
context = build_context("user-123", tier="premium")
config = get_completion_config(ai_client, "content-assistant", context)

if config.enabled:
    model = config.model.name
    messages = config.messages
    tracker = config.tracker
    print(f"[OK] Config 'content-assistant' loaded - model: {model}")
else:
    print(f"[WARNING] Config 'content-assistant' is disabled or not found")

[OK] Config 'content-assistant' loaded - model: gpt-4


---
## Verify: Completion Configs

In [36]:
# Helper to print config details
def print_config(config, name="Config"):
    print(f"\n{'='*50}")
    print(f"{name}:")
    print(f"{'='*50}")
    print(f"  Type: {type(config).__name__}")
    print(f"  Enabled: {config.enabled}")
    if hasattr(config, 'model') and config.model:
        print(f"  Model: {config.model.name}")
    if hasattr(config, 'provider') and config.provider:
        print(f"  Provider: {config.provider.name}")
    if hasattr(config, 'messages') and config.messages:
        print(f"  Messages: {len(config.messages)}")
    if hasattr(config, 'instructions') and config.instructions:
        display = f"{config.instructions[:100]}..." if len(config.instructions) > 100 else config.instructions
        print(f"  Instructions: {display}")
    if hasattr(config, 'tracker') and config.tracker:
        print(f"  Tracker: {type(config.tracker).__name__}")
    print(f"{'='*50}")

# Test completion config: content-assistant
print("=== Testing completion config: content-assistant ===")
test_context = build_context("verify-user", brand="TestBrand")
completion_config = get_completion_config(ai_client, "content-assistant", test_context, {"brand": "TestBrand", "content_request": "Write a tagline"})
print_config(completion_config, "content-assistant")

# Verify config is enabled (loaded from LaunchDarkly)
assert completion_config.enabled, "Expected config to be enabled!"
print(f"\n[OK] Successfully loaded config from LaunchDarkly")

=== Testing completion config: content-assistant ===

content-assistant:
  Type: AICompletionConfig
  Enabled: True
  Model: gpt-4
  Provider: 
  Messages: 2
  Tracker: LDAIConfigTracker

[OK] Successfully loaded config from LaunchDarkly


---
## Consuming Agent Configs
From: `SKILL.md` lines 128-151

In [37]:
from typing import Dict
from ldai.client import AIAgentConfigDefault

# Register fallback configs for specific agent config keys that need fallbacks
agent_fallback_configs: Dict[str, AIAgentConfigDefault] = {}

def get_agent_config(ai_client, config_key: str, context, variables: dict = None):
    """Get agent-mode AI Config with optional fallback."""
    fallback = agent_fallback_configs.get(config_key, AIAgentConfigDefault(enabled=False))
    config = ai_client.agent_config(config_key, context, fallback, variables or {})
    return config

# Test with agent config created by aiconfig-create cookbook
context = build_context("user-123")
config = get_agent_config(ai_client, "support-agent", context)

if config.enabled:
    instructions = config.instructions
    model_name = config.model.name
    tracker = config.tracker
    print(f"[OK] Agent config 'support-agent' loaded - model: {model_name}")
else:
    print(f"[WARNING] Agent config 'support-agent' is disabled or not found")

[OK] Agent config 'support-agent' loaded - model: gpt-4


---
## Verify: get_agent_config()

In [38]:
# Test agent config: support-agent
print("=== Testing agent config: support-agent ===")
test_context = build_context("agent-verify-user")
agent_config = get_agent_config(
    ai_client,
    "support-agent",
    test_context,
    {"company_name": "TestCorp", "support_priority": "high"}
)

print_config(agent_config, "support-agent")

# Verify config is enabled
assert agent_config.enabled, "Expected config to be enabled!"
print(f"\n[OK] Successfully loaded agent config from LaunchDarkly")

=== Testing agent config: support-agent ===

support-agent:
  Type: AIAgentConfig
  Enabled: True
  Model: gpt-4
  Provider: 
  Instructions: You are a helpful customer support agent.

Your responsibilities:
- Answer customer questions
- Reso...
  Tracker: LDAIConfigTracker

[OK] Successfully loaded agent config from LaunchDarkly


---
## Fresh Configs Per Request
From: `SKILL.md` lines 157-174

In [39]:
class DynamicAIClient:
    """Fetch fresh config for every request - never cache configs."""
    def __init__(self, ai_client, config_key: str):
        self.ai_client = ai_client
        self.config_key = config_key

    def generate(self, prompt: str, user_id: str, **user_attributes):
        """Get fresh config for each request."""
        context = build_context(user_id, **user_attributes)
        config = get_completion_config(self.ai_client, self.config_key, context)
        return config  # Process with this config

# Test with content-assistant config
client = DynamicAIClient(ai_client, "content-assistant")
config1 = client.generate("Hello", "user-123", tier="free")
config2 = client.generate("Hello", "user-456", tier="premium")

---
## Verify: DynamicAIClient

In [40]:
# Test DynamicAIClient
dynamic_client = DynamicAIClient(ai_client, "content-assistant")

# Different users getting fresh configs
config1 = dynamic_client.generate("Hello", "user-free", tier="free")
config2 = dynamic_client.generate("Hello", "user-premium", tier="premium")

print("DynamicAIClient verification:")
print(f"  Config key: content-assistant")
print(f"  User 'user-free' config enabled: {config1.enabled}")
print(f"  User 'user-premium' config enabled: {config2.enabled}")
print(f"  Both configs are different objects: {config1 is not config2}")
print(f"[OK] DynamicAIClient verified")

DynamicAIClient verification:
  Config key: content-assistant
  User 'user-free' config enabled: True
  User 'user-premium' config enabled: True
  Both configs are different objects: True
[OK] DynamicAIClient verified


---
## Handling Multiple Configs
From: `SKILL.md` lines 178-211

In [41]:
def generate_summary(text: str, config) -> str:
    """Generate summary using the config's model and messages."""
    # Use config.model.name, config.messages, config.tracker
    # Call your LLM provider and return summary
    return f"Summary of: {text[:50]}..."

def translate_text(text: str, config) -> str:
    """Translate text using the config's model and messages."""
    # Use config.model.name, config.messages, config.tracker
    # Call your LLM provider and return translation
    return f"Translated: {text[:50]}..."

def get_multiple_configs(ai_client, user_id: str):
    """Get multiple AI Configs for different purposes."""
    context = build_context(user_id)

    configs = {
        "summarizer": get_completion_config(ai_client, "content-assistant", context),
        "translator": get_completion_config(ai_client, "content-assistant", context),
        "analyzer": get_agent_config(ai_client, "support-agent", context)
    }
    return configs

# Use configs in sequence - summarize then translate the summary
text = "This is a sample document with important information about the product."
configs = get_multiple_configs(ai_client, "user-123")

if configs["summarizer"].enabled:
    summary = generate_summary(text, configs["summarizer"])
    print(f"[OK] Generated summary: {summary}")

    # Pass summary to translator
    if configs["translator"].enabled:
        translation = translate_text(summary, configs["translator"])
        print(f"[OK] Translated summary: {translation}")

[OK] Generated summary: Summary of: This is a sample document with important informati...
[OK] Translated summary: Translated: Summary of: This is a sample document with importa...


---
## Verify: get_multiple_configs()

In [42]:
# Test get_multiple_configs
print("=== Testing get_multiple_configs ===")
multi_configs = get_multiple_configs(ai_client, "multi-config-user")

print(f"Configs retrieved: {list(multi_configs.keys())}")
for name, cfg in multi_configs.items():
    status = "[OK]" if cfg.enabled else "[DISABLED]"
    model = cfg.model.name if cfg.model else 'N/A'
    print(f"  {status} {name}: model={model}")
print(f"[OK] Multiple configs pattern verified")

=== Testing get_multiple_configs ===
Configs retrieved: ['summarizer', 'translator', 'analyzer']
  [OK] summarizer: model=gpt-4
  [OK] translator: model=gpt-4
  [OK] analyzer: model=gpt-4
[OK] Multiple configs pattern verified


---
## Variable Substitution
From: `SKILL.md` lines 215-238

In [43]:
# In LaunchDarkly, your instruction might be:
# "You are a {{role}} assistant for {{company}}. Focus on {{focus_area}}."

context = build_context("user-123")

# Provide variable values at runtime
config = get_agent_config(
    ai_client,
    "support-agent",
    context,
    variables={
        "company_name": "TechCorp",
        "support_priority": "billing issues"
    }
)

if config.enabled:
    # Instructions are populated with variable values
    print(f"[OK] Variable substitution working")
    print(f"Instructions preview: {config.instructions[:150]}...")
else:
    print(f"[WARNING] Config is disabled")

[OK] Variable substitution working
Instructions preview: You are a helpful customer support agent.

Your responsibilities:
- Answer customer questions
- Resolve issues efficiently
- Maintain a friendly tone
...


---
## Verify: Variable Substitution

In [44]:
# Additional verification with support-agent config
context = build_context("variable-test-user")

config_with_vars = get_agent_config(
    ai_client,
    "support-agent",
    context,
    variables={
        "company_name": "AcmeCorp",
        "support_priority": "high"
    }
)

print("Variable substitution verification:")
print(f"  Config key: support-agent")
print(f"  Variables passed: company_name=AcmeCorp, support_priority=high")
print(f"  Config enabled: {config_with_vars.enabled}")
if config_with_vars.enabled and config_with_vars.instructions:
    print(f"  Instructions (first 200 chars): {config_with_vars.instructions[:200]}...")
print(f"[OK] Variable substitution verified")

Variable substitution verification:
  Config key: support-agent
  Variables passed: company_name=AcmeCorp, support_priority=high
  Config enabled: True
  Instructions (first 200 chars): You are a helpful customer support agent.

Your responsibilities:
- Answer customer questions
- Resolve issues efficiently
- Maintain a friendly tone

Company: AcmeCorp
Priority: high...
[OK] Variable substitution verified


---
## Error Handling
From: `SKILL.md` lines 242-266

In [45]:
import logging

logger = logging.getLogger(__name__)

def call_llm(model_name: str, messages, prompt: str) -> str:
    """Stub for LLM call - replace with actual implementation."""
    return f"Response from {model_name} for: {prompt[:30]}..."

def process_with_config(ai_client, config_key: str, user_id: str, prompt: str):
    """Process request with AI Config and proper error handling."""
    context = build_context(user_id)
    config = get_completion_config(ai_client, config_key, context)

    if not config.enabled:
        logger.warning(f"Config '{config_key}' is disabled or not found")
        return None

    try:
        # Use the config to make your LLM call
        result = call_llm(config.model.name, config.messages, prompt)
        config.tracker.track_success()
        return result

    except Exception as e:
        logger.error(f"LLM call failed: {e}")
        config.tracker.track_error()
        return None

print("[OK] process_with_config() defined")

[OK] process_with_config() defined


In [46]:
# Test error handling
print("=== Testing process_with_config ===")

# Test with real config
result = process_with_config(ai_client, "content-assistant", "test-user", "Write a tagline")
if result:
    print(f"[OK] Got result: {result}")
else:
    print("[WARNING] No result returned")

# Test with nonexistent config (should handle gracefully)
result = process_with_config(ai_client, "nonexistent-config", "test-user", "Hello")
if result is None:
    print(f"[OK] Gracefully handled nonexistent config")
else:
    print(f"[INFO] Got result: {result}")

Config 'nonexistent-config' is disabled or not found


=== Testing process_with_config ===
[OK] Got result: Response from gpt-4 for: Write a tagline...
[OK] Gracefully handled nonexistent config
